In [1]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
def cargar_parametros_desde_excel(ruta_excel='parametros.xlsx'):
    """
    Carga todos los parámetros desde un archivo Excel con múltiples hojas
    
    Args:
        ruta_excel: Ruta al archivo Excel con las hojas de parámetros
        
    Returns:
        params: Diccionario con todos los parámetros cargados
    """
    params = {}
    
    # 1. Cargar parámetros generales desde la hoja "parametros"
    df_params = pd.read_excel(ruta_excel, sheet_name='parametros')
    params['parametros_generales'] = {row[0]: row[1] for _, row in df_params.iterrows()}
    
    # Convertir tipos de datos según el tipo esperado
    for key in ['almacenamiento', 'costo_plantacion', 'dias_anticipacion', 
                'min_dias_aclimatacion', 'max_dias_aclimatacion', 'dias_simulacion',
                'velocidad_camioneta', 'tiempo_maximo', 'tiempo_carga', 
                'carga_maxima', 'capacidad_maxima_transporte', 'costo_transporte','espacio_por_planta']:
        if key in params['parametros_generales']:
            if key in ['dias_anticipacion', 'min_dias_aclimatacion', 
                      'max_dias_aclimatacion', 'dias_simulacion']:
                params['parametros_generales'][key] = int(params['parametros_generales'][key])
            else:
                params['parametros_generales'][key] = float(params['parametros_generales'][key])
    
    # 2. Cargar hectáreas por polígono desde la hoja "hectareas_poligono"
    df_hectareas = pd.read_excel(ruta_excel, sheet_name='hectareas_poligono')
    params['hectareas_poligono'] = dict(zip(df_hectareas['poligono'], df_hectareas['hectareas']))
    
    # 3. Cargar demanda por especie desde la hoja "demanda_especies"
    df_demanda = pd.read_excel(ruta_excel, sheet_name='demanda_especies')
    params['demanda_especies_por_hectarea'] = dict(zip(df_demanda['especie'], 
                                                     df_demanda['demanda_por_hectarea']))
    
    # 4. Cargar costos de proveedores desde la hoja "costos_proveedores"
    df_costos = pd.read_excel(ruta_excel, sheet_name='costos_proveedores')
    
    # Extraer nombres de proveedores (de las columnas excluyendo la primera)
    params['nombres_proveedores'] = df_costos.columns[1:].tolist()
    
    # Extraer nombres de especies (de la primera columna)
    params['nombres_especies'] = df_costos.iloc[:, 0].tolist()
    
    # Extraer matriz de costos (convertir a float, eliminando la primera columna)
    params['matriz_costos'] = df_costos.iloc[:, 1:].values.astype(float)
    
    # 5. Cargar distancias entre polígonos desde la hoja "distancias_poligonos"
    df_distancias = pd.read_excel(ruta_excel, sheet_name='distancias_poligonos', header=None)
    params['distancias_poligonos'] = df_distancias.values
    
    
    # Información adicional
    print(f"Parámetros cargados correctamente del archivo Excel: {ruta_excel}")
    print(f"Se cargaron {len(params['hectareas_poligono'])} polígonos")
    print(f"Se cargaron {len(params['nombres_especies'])} especies y {len(params['nombres_proveedores'])} proveedores")
    
    return params

In [ ]:
# Cargar todos los parámetros desde archivo Excel
params = cargar_parametros_desde_excel('parametros.xlsx')

# Extraer parámetros generales
parametros_generales = params['parametros_generales']
almacenamiento = parametros_generales.get('almacenamiento')
# NUEVO: Definir espacio por planta y calcular capacidad máxima de plantas en inventario
espacio_por_planta = parametros_generales.get('espacio_por_planta')  # m2 por planta
capacidad_maxima_plantas = int(almacenamiento / espacio_por_planta)
costo_plantacion = parametros_generales.get('costo_plantacion')
dias_anticipacion = parametros_generales.get('dias_anticipacion')
min_dias_aclimatacion = parametros_generales.get('min_dias_aclimatacion')
max_dias_aclimatacion = parametros_generales.get('max_dias_aclimatacion')
dias_simulacion = parametros_generales.get('dias_simulacion')
velocidad_camioneta = parametros_generales.get('velocidad_camioneta')
tiempo_maximo = parametros_generales.get('tiempo_maximo')
tiempo_carga = parametros_generales.get('tiempo_carga')
carga_maxima = parametros_generales.get('carga_maxima')
capacidad_maxima_transporte = parametros_generales.get('capacidad_maxima_transporte')
costo_transporte = parametros_generales.get('costo_transporte')
start_polygon = int(parametros_generales.get('start_polygon'))

# Extraer otros datos
hectareas_poligono = params['hectareas_poligono']
demanda_especies_por_hectarea = params['demanda_especies_por_hectarea']
nombres_especies = params['nombres_especies']
nombres_proveedores = params['nombres_proveedores']
matriz_costos = params['matriz_costos']

# Procesar matriz de distancias
distancias_poligonos = params['distancias_poligonos']
dist_poligonos_hrs = distancias_poligonos / velocidad_camioneta
dist_poligonos_hrs = np.round(dist_poligonos_hrs, 2)

# Determinar el número total de polígonos
NUM_POLYGONS = min(dist_poligonos_hrs.shape[0], max(hectareas_poligono.keys()) + 1)  # +1 para incluir todos los polígonos
START_POLYGON = start_polygon

# Imprimir parámetros cargados
print("\n--- PARÁMETROS DE SIMULACIÓN (CARGADOS DE EXCEL) ---")
print(f"Días de simulación: {dias_simulacion}")
print(f"Capacidad máxima de transporte: {capacidad_maxima_transporte}")
print(f"Días de anticipación para pedidos: {dias_anticipacion}")
print(f"Días mínimos de aclimatación: {min_dias_aclimatacion}")
print(f"Capacidad máxima de plantas en inventario: {capacidad_maxima_plantas}")
print(f"Días máximos de aclimatación: {max_dias_aclimatacion}")
print(f"Capacidad de carga por ruta: {carga_maxima}")
print(f"Tiempo máximo por ruta: {tiempo_maximo} horas")

Parámetros cargados correctamente del archivo Excel: parametros.xlsx
Se cargaron 19 polígonos
Se cargaron 9 especies y 4 proveedores

--- PARÁMETROS DE SIMULACIÓN (CARGADOS DE EXCEL) ---
Días de simulación: 30
Capacidad máxima de transporte: 8000.0
Días de anticipación para pedidos: 1
Días mínimos de aclimatación: 3
Capacidad máxima de plantas en inventario: 10000
Días máximos de aclimatación: 7
Capacidad de carga por ruta: 524.0
Tiempo máximo por ruta: 6.0 horas

--- PARÁMETROS OBTENIDOS DE EXCEL ---
almacenamiento = 400.0
espacio_por_planta = 0.04
capacidad_maxima_plantas = 10000
costo_plantacion = 20.0
dias_anticipacion = 1
min_dias_aclimatacion = 3
max_dias_aclimatacion = 7
dias_simulacion = 30
velocidad_camioneta = 40.0
tiempo_maximo = 6.0
tiempo_carga = 0.5
carga_maxima = 524.0
capacidad_maxima_transporte = 8000.0
costo_transporte = 4500.0
start_polygon = 6
NUM_POLYGONS = 19
START_POLYGON = 6
(9, 4)
(19, 19)
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18